In [9]:
import xarray as xa
import numpy as np
from numba import njit
import joblib

In [10]:
import matplotlib as plt

In [11]:
datafile = './utility/aggregate_be.nc'
data = xa.open_dataset(datafile, decode_times=False)

In [14]:
# print(data)

def setNxNY(Nx, Ny):
    global N
    N = np.array([Nx,Ny])

    
    
u = data.u[:,0,0:350,0:700]
v = data.v[:,0,0:350,0:700]

x = data.X[0:700] * 1000 #convert to meter
y = data.Y[0:350] * 1000

print(data.X.values.shape)
time_res = data.time.values[1] - data.time.values[0]
x_res = x.values[1] - x.values[0]
y_res = y.values[1] - y.values[0]    
@njit(parallel=True)
def f(X, t):
    x_0 = ((X - x.values[0]) / x_res).astype(np.int)
    t_0 = int(t/time_res)
    
    x_d = ((X - x.values[0]) / x_res - x_0)
    t_d = t/time_res - t_0
    
    ret = np.empty((2, len(X[0,:])))
    
    c00 = u[t_0, x_0[1,:], x_0[0,:]] * (1 - x_d[0,:]) + u[t_0 + 1, x_0[1,:], x_0[0,:]] * x_d[0,:]
    c01 = u[t_0, x_0[1,:], x_0[0,:] + 1] * (1 - x_d[0,:]) + u[t_0 + 1, x_0[1,:], x_0[0,:] + 1] * x_d[0,:]
    c10 = u[t_0, x_0[1,:] + 1, x_0[0,:]] * (1 - x_d[0,:]) + u[t_0 + 1, x_0[1,:] + 1, x_0[0,:]] * x_d[0,:]
    c11 = u[t_0, x_0[1,:] + 1, x_0[0,:] + 1] * (1 - x_d[0,:]) + u[t_0 + 1, x_0[1,:] + 1, x_0[0,:] + 1] * x_d[0,:]
    
    c0 = c00 * (1 - x_d[1,:]) + c10 * x_d[1,:]
    c1 = c01 * (1 - x_d[1,:]) + c11 * x_d[1,:]
    
    ret[0,:] = c0 * (1 - t_d) + c1 * t_d
    
    c00 = v[t_0, x_0[1,:], x_0[0,:]] * (1 - x_d[0,:]) + v[t_0 + 1, x_0[1,:], x_0[0,:]] * x_d[0,:]
    c01 = v[t_0, x_0[1,:], x_0[0,:] + 1] * (1 - x_d[0,:]) + v[t_0 + 1, x_0[1,:], x_0[0,:] + 1] * x_d[0,:]
    c10 = v[t_0, x_0[1,:] + 1, x_0[0,:]] * (1 - x_d[0,:]) + v[t_0 + 1, x_0[1,:] + 1, x_0[0,:]] * x_d[0,:]
    c11 = v[t_0, x_0[1,:] + 1, x_0[0,:] + 1] * (1 - x_d[0,:]) + v[t_0 + 1, x_0[1,:] + 1, x_0[0,:] + 1] * x_d[0,:]
    
    c0 = c00 * (1 - x_d[1,:]) + c10 * x_d[1,:]
    c1 = c01 * (1 - x_d[1,:]) + c11 * x_d[1,:]
    
    ret[1,:] = c0 * (1 - t_d) + c1 * t_d
    return ret

# @njit(parallel=True)
# def doublegyre(X, t, A, e, w):
#     a = e * np.sin(w*t)
#     b = 1 - 2.*e*np.sin(w*t)
#     f = a*X[0,:]**2 + b*X[0,:]
#     v = np.empty(X.shape)
#     v[0,:] = -np.pi*A*np.sin(np.pi*f) * np.cos(np.pi*X[1,:])                    # x component of velocity
#     v[1,:] =  np.pi*A*np.cos(np.pi*f) * np.sin(np.pi*X[1,:]) * (2.*a*X[0,:] + b) # y component of velocity
#     return v


# # Wrapper function to pass to integrator
# # X0 is a two-component vector [x, y]
# @njit
# def f(X, t):
#     # Parameters of the velocity field
#     A = A_       # A
#     e = e_      # epsilon
#     w = 2.*np.pi/10.  # omega
#     return doublegyre(X, t, A, e, w)


(751,)
